In [1]:
import folium
import wget
import pandas as pd

In [2]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head()


100% [............................................................] 2476 / 2476

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
#LET'S SEE WHICH SITE HAS LARGEST SUCCESS COUNT:

sites_dicts = []
sites_dicts.append({'label': 'All Sites', 'value': 'ALL'})
for site in spacex_df.groupby(['Launch Site'])['class'].count().sort_values(ascending=False).index.to_list():
    dizi = {'label':site,'value':site}
    sites_dicts.append(dizi)
sites_dicts


[{'label': 'All Sites', 'value': 'ALL'},
 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}]

In [ ]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
        # TASK 1: Add a dropdown list to enable Launch Site selection
        # The default select value is for ALL sites
        dcc.Dropdown(id='site-dropdown',
                options=[x for x in sites_dicts], #definito sopra,
                value='ALL',
                placeholder="place holder here",
                searchable=True
                ),
        html.Br(),

        # TASK 2: Add a pie chart to show the total successful launches count for all sites
        # If a specific launch site was selected, show the Success vs. Failed counts for the site
        html.Div(dcc.Graph(id='success-pie-chart')),
        html.Br(),

        html.P("Payload range (Kg):"),
        # TASK 3: Add a slider to select payload range
        dcc.RangeSlider(id='payload-slider',
                        min=0, max=10000, step=1000,
                        marks={0: '0',
                               1250: '1250',
                               2500: '2500',
                               3750: '3750',
                               5000: '5000',
                               6250: '6250',
                               7500: '7500',
                               8750: '8750',
                               10000: '10000'},
                        value=[min_payload, max_payload]),
        # TASK 4: Add a scatter chart to show the correlation between payload and launch success
        html.Div(dcc.Graph(id='success-payload-scatter-chart')),
        ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        # return the outcomes piechart for a selected site
        fig = px.pie(filtered_df, names='class',
        title='Total Success Launches for Site {}'.format(entered_site))
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")])
def get_scatter_plot(entered_site,entered_payload_range):
    spacex_dff = spacex_df[spacex_df["Payload Mass (kg)"].between(entered_payload_range[0],entered_payload_range[1],inclusive=True)]
    if entered_site == 'ALL':
        fig = px.scatter(spacex_dff, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                         title="Correlation between Payload and Class for All sites")
        return fig
    else:
        filtered_dff = spacex_dff[spacex_dff['Launch Site']==entered_site]
        # return the outcomes piechart for a selected site
        fig = px.scatter(spacex_dff, x="Payload Mass (kg)", y="class", color="Booster Version Category",title='Correlation between Payload and Class for Site {}'.format(entered_site))
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
